### THIS FILE CONTAINS CONFIDENTIAL DATA, ONLY RAW CODE HAS BEEN UPLOADED. INDIVIDUAL CELL OUTPUTS HAVE BEEN OMITTED. ANONYMIZATION KEY AND RAW VIEWERSHIP DATA WILL NOT BE LOADED TO DATABASE AND UNAVAILABLE. THIS CODE WILL DOCUMENT THE CLEANING FOR THE VIEWERSHIP DATA

In [ ]:
### Data Processing Into DataFrames
# Import dependancies
import pandas as pd
from datetime import datetime, timedelta, date
from pathlib import Path


In [ ]:
# Merging Minute level aggregation into one table

#Determine input date range
start_date = "2021-02-18"
end_date = "2022-11-06"

#string to datetime
start_date_dt = datetime.strptime(start_date, '%Y-%m-%d').date()
end_date_dt = datetime.strptime(end_date, '%Y-%m-%d').date()

In [ ]:
# define a function that will read all the csv files and combine them into one dataframe.

def minute_data_aggregation_condenser(start, end):
    current_date = start
    combined_df = pd.DataFrame(pd.read_csv(f"Resources/minutelevelsessionaggregations-qwest-{start_date_dt}.csv"))
    counter = 1

    while current_date <= end:
        try:
            current_date = current_date + timedelta(days=1)
            current_data = pd.read_csv(f"Resources/minutelevelsessionaggregations-qwest-{current_date}.csv")
            current_df = pd.DataFrame(current_data)
            combined_df = pd.concat([combined_df, current_df])
            counter += 1
        except:
            current_date = current_date + timedelta(days=1)
    else:
        print(f"Data Merge Complete, {counter} files have been merged into a dataframe and exported as merged_aggregate_data{date.today()}.csv")
        return combined_df

combined_df = minute_data_aggregation_condenser(start_date_dt, end_date_dt)
combined_df.head()

In [ ]:
#check df length
print(len(combined_df))

In [ ]:
# Check columns in df
combined_df.columns


In [ ]:
#Extra column named channel;time;content_id;country;total_sessions;total_session_duration_seconds indicates error
# Check that column
len(combined_df.columns)
combined_df['channel;time;content_id;country;total_sessions;total_session_duration_seconds'].value_counts()

In [ ]:
# remove all rows containing bad data
combined_df = combined_df[pd.isnull(combined_df['channel;time;content_id;country;total_sessions;total_session_duration_seconds'])]

# Column was semicolon separated rather than comma separated on 2022-06-26
semicolon_data = pd.read_csv("Resources/minutelevelsessionaggregations-qwest-2022-06-26.csv", sep=';')
semicolon_df = pd.DataFrame(semicolon_data)
semicolon_df.head()

#combine with complete dataframe
combined_df=pd.concat([combined_df, semicolon_df])

#remove 'channel;time;content_id;country;total_sessions;total_session_duration_seconds' column
combined_df = combined_df.drop(columns=['channel;time;content_id;country;total_sessions;total_session_duration_seconds'])
combined_df.head()



In [ ]:
#check length of df to make sure it matches with previous 
print(len(combined_df))

In [ ]:
# import anonymization key
anon_key = pd.read_csv("Resources/Anonymization Keys.csv")
anon_key_df = pd.DataFrame(anon_key)
anon_key_df

anon_key_op_df = anon_key_df[['Operator', 'anonymization key']]
anon_key_chan_df = anon_key_df[['Channel', 'Anonymization key (Genre)']]
anon_key_chan_df = anon_key_chan_df.dropna()

In [ ]:
#match channel with operator ### EXPECTED TIME 349minutes
def string_parser_OPS (string):
    for ops in anon_key_op_df['Operator']:
        if string.str.contains(ops.lower()).any():
            return anon_key_op_df.loc[anon_key_op_df['Operator']== ops, 'anonymization key'].item()

combined_df["Operator"] = combined_df[['channel']].apply(string_parser_OPS, axis =1, result_type='expand')

In [ ]:
#match channel with channel ### EXPECTED TIME 244minutes
def string_parser_CHAN (string):
    for ops in anon_key_chan_df['Channel']:
        if string.str.contains(ops.lower()).any():
            return anon_key_chan_df.loc[anon_key_chan_df['Channel']== ops, 'Anonymization key (Genre)'].item()

combined_df["Channel"] = combined_df[['channel']].apply(string_parser_CHAN, axis =1, result_type='expand')

In [ ]:
# Output new dataframe to CSV
filepath = Path(f'Resources/cleaned_merged_aggregate_data-{date.today()}.csv')
combined_df.to_csv(filepath)